<center>
<img src="../../img/ods_stickers.jpg">
## Open Machine Learning Course
<center>
Author: Yury Kashnitsky, Data Scientist at Mail.Ru Group

This material is subject to the terms and conditions of the license [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Free use is permitted for any non-comercial purpose with an obligatory indication of the names of the authors and of the source.

# <center> Assignment #10. Solution
## <center> Gradient boosting

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

/Users/y.kashnitsky/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module '_catboost' does not match runtime version 3.6
  return f(*args, **kwds)


**Read the data**

In [2]:
train_df = pd.read_csv('../../data/flight_delays_train.csv')

In [3]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test_df = pd.read_csv('../../data/flight_delays_test.csv')

In [5]:
test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


**Create only one feature - “flight”**

In [6]:
train_df['flight'] = train_df['Origin'] + '-->' + train_df['Dest']
test_df['flight'] = test_df['Origin'] + '-->' + test_df['Dest']

**Remember the indexes of categorical features**

In [7]:
categ_feat_idx = np.where(train_df.drop('dep_delayed_15min', axis=1).dtypes == 'object')[0]
categ_feat_idx

array([0, 1, 2, 4, 5, 6, 8])

**Allocate a hold-out set**

In [8]:
X_train = train_df.drop('dep_delayed_15min', axis=1).values
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test_df.values

In [9]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [10]:
ctb = CatBoostClassifier(random_seed=17)

**Train Catboost without setting hyperparameters, passing only the indexes of categorical features.**

In [11]:
%%time
ctb.fit(X_train_part, y_train_part,
        cat_features=categ_feat_idx)

CPU times: user 5min 19s, sys: 26.3 s, total: 5min 46s
Wall time: 55.1 s


In [12]:
ctb_valid_pred = ctb.predict_proba(X_valid)[:, 1]

**We got almost 0.75 ROC AUC on a hold-out set.**

In [13]:
roc_auc_score(y_valid, ctb_valid_pred)

0.74978752558569872

**Train on the whole train set, make prediction on the test set. We got 0.73008 in the competition.**

In [14]:
%%time
ctb.fit(X_train, y_train,
        cat_features=categ_feat_idx)

CPU times: user 6min 37s, sys: 23.7 s, total: 7min
Wall time: 1min 3s


In [15]:
ctb_test_pred = ctb.predict_proba(X_test)[:, 1]

In [17]:
sample_sub = pd.read_csv('/Users/y.kashnitsky/Documents/Machine_learning/org_mlcourse_open/private/competitions/kaggle_flights/sample_submission.csv', index_col='id')
sample_sub['dep_delayed_15min'] = ctb_test_pred
sample_sub.to_csv('ctb_pred.csv')

In [18]:
sample_sub.head()

,dep_delayed_15min
id,
0,0.038922
1,0.059309
2,0.042454
3,0.244527
4,0.291769
